I make a code that loads the submission file and show the top 8 team's tournament.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Just copying this code block, you can shows your tournament result.
# You need to change result db name to submision.

DATA_PATH = '../input/mens-march-mania-2022/MDataFiles_Stage2/'

# Reference: Sam Seljan'code at discussion https://www.kaggle.com/c/mens-march-mania-2022/discussion/312777
seeds = pd.read_csv(DATA_PATH + 'MNCAATourneySeeds.csv')
seeds = seeds[seeds.Season==2022]

slots = pd.read_csv(DATA_PATH + 'MNCAATourneySeedRoundSlots.csv')

slot_seeds = pd.merge(seeds,slots,on='Seed')
game_pairs = pd.merge(slot_seeds[['TeamID','GameSlot','GameRound']],
slot_seeds[['TeamID','GameSlot','GameRound']], on = ['GameSlot'])
game_pairs['Round'] = game_pairs['GameRound_x']
game_pairs = game_pairs[game_pairs['TeamID_x'] > game_pairs['TeamID_y']].copy()
game_pairs['ID'] = '2022_' + game_pairs['TeamID_y'].astype(str) + '_' + game_pairs['TeamID_x'].astype(str)

fist_four_index = game_pairs[game_pairs['GameSlot'].str[0] != 'R'].index
# By handle as R1?5 to set middle.
first_four_game_id = ['2022_1231_1461', '2022_1323_1353', '2022_1394_1411', '2022_1136_1460']
game_pairs.loc[6326, 'GameSlot'] = 'R1W4'
game_pairs.loc[6874, 'GameSlot'] = 'R1X5'
game_pairs.loc[8578, 'GameSlot'] = 'R1Y0'
game_pairs.loc[9126, 'GameSlot'] = 'R1Z0'

game_pairs = game_pairs.sort_values(by='GameSlot')

game_ids = []
for x, y in zip(game_pairs['TeamID_x'], game_pairs['TeamID_y']):
    if not y in game_ids:
        game_ids.append(y)
    if not x in game_ids:
        game_ids.append(x)
    
survived_pairs = pd.DataFrame([])
survived_ids = []

def drop_lose_team(pairs):
    global survived_pairs, survived_ids
    survived_pairs = survived_pairs.drop(pairs.index)
    for index, row in pairs.iterrows():
        if row['Pred'] > 0.5:
            drop_team = row['TeamID_x']
        else:
            drop_team = row['TeamID_y']
        survived_pairs = survived_pairs[survived_pairs['TeamID_x']!=drop_team]
        survived_pairs = survived_pairs[survived_pairs['TeamID_y']!=drop_team]
        survived_ids.remove(drop_team)


def show_tournament(SHOW_DEPTH, submission):
    global game_pairs, game_ids, survived_pairs, survived_ids, fist_four_index
    
    # Connect game_pairs to submission.
    survived_pairs = game_pairs.merge(submission[['ID', 'Pred']], on='ID', how='left')
    survived_pairs = survived_pairs.sort_values(by='GameSlot')
    survived_ids = game_ids.copy()
    
    # First Four
    first_four = pd.concat([survived_pairs[survived_pairs['GameSlot'].str[1] == '1'][survived_pairs['ID'] == game_id] for game_id in first_four_game_id])
    pairs = first_four.drop_duplicates(subset='ID', keep='first')
    drop_lose_team(pairs)

    result = []
    for i in range(1,6):
        pairs = survived_pairs[survived_pairs['GameSlot'].str[1] == str(i)]
        drop_lose_team(pairs)
        if i > 6-SHOW_DEPTH:
            result.append(survived_ids.copy())

    last_pair = survived_pairs.iloc[0]
    if last_pair['Pred'] > 0.5:
        result.append([last_pair['TeamID_y']])
    else:
        result.append([last_pair['TeamID_x']])

    # Add show_text list to show
    team = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage2/MTeams.csv')
    show_text = [''] * (2**len(result)-1)
    depth = 0
    for depth in range(len(result)):
        for i in range(len(show_text)):
            position = i - (2**depth) + 1
            if position % 2**(depth+1) == 0:
                team_id = result[depth][int(i / (2**(depth+1)))]
                show_text[i] += team[team['TeamID'] == team_id]['TeamName'].iloc[0][:9].ljust(10)
                if position % 2**(depth+2) == 0:
                    show_text[i] += '━┓  '
                else:
                    show_text[i] += '━┛  '
            else:
                if position % (2**(depth+2)) == 2**(depth):
                    show_text[i] += ' '.ljust(11) + '┃━ '
                elif (position % (2**(depth+2))) < 2**(depth+1):
                    show_text[i] += ' '.ljust(11) + '┃  '
                else:
                    show_text[i] += ' '.ljust(14)

    # Showing tornament
    for text in show_text:
        print(text[:-4])

In [ ]:
# Please load your submission file here
# Or copy above code to your code
SUB_PATH = '../input/mmlmfirstsubmission/submission.csv'

submission = pd.read_csv(SUB_PATH)
submission.head(5)

In [ ]:
# Elite Eight Tournament
show_tournament(4,submission)

In [ ]:
# 64 Teams Tournament
show_tournament(6,submission)